<a href="https://colab.research.google.com/github/leotuni/Nicolas-stuff/blob/master/Pulse_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PULSE measurment Notebook 

The Paper from Ilaria: [Passive radiofrequency x-ray dosimeter tag based on flexible radiation-sensitive oxide field-effect transistor](https://advances.sciencemag.org/content/4/6/eaat1825#aff-1)

Supporting material of the Paper: 
[Link](https://advances.sciencemag.org/content/advances/suppl/2018/06/25/4.6.eaat1825.DC1/aat1825_SM.pdf) 

# CODE 

## CODE: imports and functions for value extraction

I have organized the notebook with all imports/class/fucntions at the top. In the main the funcitons are just callled. 

In [ ]:
#@title Library imports and Mount Drive 

import pandas as pd 
import os #to change directory
import glob #for file list 
import re
import numpy as np
from scipy.stats import linregress

#plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt



from google.colab import drive
drive.mount('/content/drive')

In [17]:
#@title Class: clean { form-width: "20px" }

"Class clean is used to Import the data and return it in a cleaner format"

class Clean:
  def __init__(self, file_list):
    '''
    Pass it a filelist, It creates a self.file_list for reference,
    and a self.clean: a list of dataframes imported and formatted. 
    
    
    '''
    
    self.file_list = file_list
    self.clean=list(self.import_clean(file_) for file_ in self.file_list)
    
  def import_clean(self, file_): 
    '''
    Imports the dataframe - removes the last column time (duplicate) - makes column values numeric - 
    '''

    #begin cleaing file
    df=pd.read_csv(file_, sep='\t', header=None)


    "Filter"
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True) #drop the last column since it is = to col2 (time)

    #filter out all non headers 
    df = df.apply(pd.to_numeric, errors='coerce').fillna(df) #change floats from str to floats
    numeric=~pd.Series([isinstance(key, str) for key in df[0]]) #creates the mask of values to headers to remove headers

    #sample and iteration 
    iterations= self.extract_from_header(df, col=1, marker='=', to_remove='iMeasureCycle=')
    sample= self.extract_from_header(df, col=2, marker='=', to_remove='iMux=')
    time= self.extract_from_header(df, col=3, marker='=', to_remove='time=')

    # col= #location of column names 
    df.rename(columns=df.iloc[1], inplace=True)#rename columns after the 2nd row

    #Add the new columns
    df['iterations']=iterations #add the iteration 
    df['sample']=sample
    df['time']=time

    df=df.loc[numeric] #remove all headers
    df.reset_index(drop=True,inplace=True) #reset the idex
    df = df.apply(pd.to_numeric, errors='coerce') #make everything numbers now that htey are

    return df
  
  def extract_from_header(self, df_, col, marker='=', to_remove='iMeasureCycle='): 

    '''
    Extracts the information from the headers for example: 
    for iMux  
    returns the row projected to the length of the dataframe.
    '''

    var=df_[col].where(df_[col].str.contains(marker).fillna(False)).ffill() #iteration based on the iMeasureCycle value 
    var=[float(key.replace(to_remove,'')) for key in var] #get the interger for it
    return var
    



In [18]:
#@title Fun: Vth, mob_sat, Onoff, subthreshold_swing, Vth_mob_extraction_vs_iteration { form-width: "30px" }

def Vth_sat(a, b):
  '''
  Pass it VG, sqrt(ID) 
  Calculate the Vth using the intercept of x method. 
  Takes 2 lines and returns the x intercept 
  '''
  res=linregress(a, b)
  slope, intercept =res[0], res[1]

  return (-intercept/slope)


def mob_sat(VG, ID, C, W=320e-4, L=20e-4):
  '''
  Pass it (VG, ID)
  Satmob--> u =m_sat^2 * (2L/W)*1/C

  It linregression (VG, np.sqrt(ID))
  '''

  res=linregress(VG, np.sqrt(abs(ID)))
  msat, intercept =res[0], res[1]
  
  us=(msat**2)*(2*L/W)/C
  return us

def on_off(a,b, limit=1e-14): 
  '''
  Pass it (max(absID), min(absID))
  The limit represents a dection limit of the machine '''
  if a==0: 
    a=limit
  if b==0: 
    b=limit 
  return (a/b)

def subthreshold_swing (VG, ID):
  '''
  Pass it (VG, ID)
  computes dlogIDdVG
  returns 1/dlogIDdVG
  '''
  dVG=np.diff(VG)
  dlogID=np.diff(np.log(abs(ID)))
  mask = dVG!=0 #removes diff=0, cause a/0 is undefined
  dlogIDdVG= dlogID[mask]/dVG[mask]

  return (1/max(dlogIDdVG))
    

def Vth_mob_extraction_vs_iteration(df_, ID_lim=0.002, C=3.3e-8, W=320e-4, L=20e-4):
  '''
  For a SINGLE dataframe, for each iteration it extacts: 
  Vth_sat 
  Mob_sat
  time
  iteration
  and returns them as a dataframe. 
  
  ID_lim is used to identify the linear region for VTh, mob linear fit. C=capacitance.


  '''
  Vth, Mob_sat, iteration, time = ([] for i in range(4))
  for key, grp in df_.groupby(['iterations']):

     #append atime and iteration
    time.append(grp['time'].iloc[0])
    iteration.append(grp['iterations'].iloc[0])

     #limit dataframe to linear region for Vth, Mob extraction 
    mask_ID_lim = (np.sqrt(abs(grp['IDrain(A)']))>ID_lim)
    grp=grp[mask_ID_lim]

     #Vth extraction
    Vth.append( Vth_sat(grp['#VGate(V)'], np.sqrt(grp['IDrain(A)']))) #uses SQRT 
     #mob extactinon
    Mob_sat.append( mob_sat(grp['#VGate(V)'], (grp['IDrain(A)']), C, W, L)) # don't uses SQRT 
   #make a dataframe and return it 
  return (pd.DataFrame({'iteration':iteration, 'time(s)': time, 'Vth_sat': Vth, 'Mob_sat':Mob_sat  }))




In [19]:
#@title Fun PLOT: Vth-Mob vs time { form-width: "30px" }

def Plot_Pulse_param (df_):
  '''
  pass it only the first iteration
  plots VG-VD-absID vs time ID is given its own axis.

  The plotting DOESN'T chage the dataframe, so mistakes in Vth are plotted. 
  '''
  fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Vth vs time
  fig.add_trace(
        go.Scatter(x=df_["time(s)"], y=df_["Vth_sat"], name='Vth_sat',
                   hovertext=df_['iteration'] ,
                   hovertemplate =
                   '<b>Vth_sat<b>: %{y:.2f} [V]'   +
                   '<br><b>time</b>: %{x: .1f} [s]<br>'+
                   '<b>Iter<b>: %{hovertext}'),
                 secondary_y=False 
    )

   #Mob vs time
  fig.add_trace(
        go.Scatter(x=df_["time(s)"], y=df_["Mob_sat"], name='Mob_sat',
                   hovertext=df_['iteration'] ,
                   hovertemplate =
                   '<b>Mob_sat<b>: %{y:.2f} [cm^2*Vsomthing]'   +
                   '<br><b>time</b>: %{x: .1f} [s]<br>'+
                   '<b>Iter<b>: %{hovertext}'),
        secondary_y=True
    )
  
  #  # Add figure title
  fig.update_layout( title_text="Mob and Vth vs Time", titlefont=dict(size=30), title_x=0.5)

   # Set x,y-axis title
  fig.update_xaxes(title_text="Time[s]", titlefont=dict(size=20))
  fig.update_yaxes( title_text="Vth[V]", secondary_y=False, titlefont=dict(size=20))
  fig.update_yaxes( title_text="Mob[units]", secondary_y=True, titlefont=dict(size=20))
  fig.update_layout(
      autosize=False,
      width=800,
      height=600,
      margin=dict(
          l=50,
          r=50,
          b=100,
          t=100,
          pad=4
      ),
      #paper_bgcolor="LightSteelBlue",
  )

  fig.show()

## CODE: supporting material 

In [20]:
#@title Supporting material PLOT: Pulse param { form-width: "30px" }

def Plot_Pulse_param (df_):
  '''
  pass it only the first iteration
  plots VG-VD-absID vs time ID is given its own axis.
  '''
  fig = make_subplots(specs=[[{"secondary_y": True}]])
    # VG vs time
  fig.add_trace(
        go.Scatter(x=df_["time(s)"], y=df_["#VGate(V)"], name='VG', mode='markers' , marker_color='black'),
        secondary_y=False
    )
    #  VD vs time
  fig.add_trace(
        go.Scatter(x=df_["time(s)"], y=df_["VDrain(V)"], name='VD', mode='markers', marker_symbol='square', marker_color='blue'),
        secondary_y=False
    )
    # ID vs time
  fig.add_trace(
        go.Scatter(x=df_["time(s)"], y=abs(df_["IDrain(A)"]), name='absID', mode='markers', marker_symbol='star', marker_color='red'),
        secondary_y=True
    )
  
   # Add figure title
  fig.update_layout( title_text="PULSE param - VG/VD/absID vs Time", titlefont=dict(size=30), title_x=0.5)

   # Set x,y-axis title
  fig.update_xaxes(title_text="Time[s]", titlefont=dict(size=20))
  fig.update_yaxes( title_text="VG[V]", secondary_y=False, titlefont=dict(size=20))
  fig.update_yaxes( title_text="absID[A]", secondary_y=True, titlefont=dict(size=20))
  fig.update_layout(
      autosize=False,
      width=600,
      height=600,
      margin=dict(
          l=50,
          r=50,
          b=100,
          t=100,
          pad=4
      ),
      #paper_bgcolor="LightSteelBlue",
  )

  fig.show()


In [21]:
#@title Supporitng material PLOT: Vths extraction { form-width: "30px" }

def Plot_Pulse_VTh_extraction(df_, ID_lim=0.002):
  '''
  Pass it df_ 1 iteration. 
  ID_lim drain min value to be considered linear region

  Plots ID vs VG - the linear fit - Vth point.

  '''
   #mask of linear region
  mask_ID_lim = (np.sqrt(abs(df_['IDrain(A)']))>ID_lim)
   #linear regression part 
  res=linregress(df_[mask_ID_lim]["#VGate(V)"], np.sqrt(abs(df_[mask_ID_lim]["IDrain(A)"])))
  slope, intercept =res[0], res[1]
  x_grid = np.linspace(0,np.max(df_[mask_ID_lim]["#VGate(V)"]),20) #grid to plot slope, intercept

  #make figure
  fig = make_subplots(specs=[[{"secondary_y": True}]])
   # ID vs VG 
  fig.add_trace(
        go.Scatter(x=df_["#VGate(V)"], y=np.sqrt(abs(df_["IDrain(A)"])), name='sqrt(absID)', mode='markers'),
        secondary_y=False
    )
    # Fit 
  fig.add_trace(
        go.Scatter(x=x_grid, y=x_grid*slope+intercept, name='Fit for sqrt(abdID)>0.002'),
        secondary_y=False
    )
    # Annotation of Vth point
  fig.add_annotation(
          x=-intercept/slope,
          y=0,
          xref="x",
          yref="y",
          text=f"Vth = {-intercept/slope} ",
          showarrow=True,
          secondary_y=True,
          font=dict(
              family="Courier New, monospace",
              size=16,
              color="#ffffff"
              ),
          align="center",
          arrowhead=2,
          arrowsize=1,
          arrowwidth=2,
          arrowcolor="#636363",
          ax=20,
          ay=-30,
          bordercolor="#c7c7c7",
          borderwidth=2,
          borderpad=4,
          bgcolor="#ff7f0e",
        
          )
  fig.update_layout( title_text="Vth extraction from Pulse", titlefont=dict(size=30), title_x=0.5)

  # Set x,y-axis title
  fig.update_xaxes(title_text="VG [V]", titlefont=dict(size=20))
  fig.update_yaxes( title_text="sqrt(absID) [A]", secondary_y=False, titlefont=dict(size=20))
  # fig.update_yaxes( title_text="Time[s]", secondary_y=True, titlefont=dict(size=20))


  fig.show()


#MAIN

## Importing the Data


All the "PULSE" measurments are stored in a single folder with the data and \_params_.
The directory indicates the Folder location in Google Drive. 
filter_out: '\_params_' gets rids of all '\_params_' files the list of files to import.

Out: 

*  Pulse.Clean stores a list of properely formatted dataframes 

*  Pulse.file_list stores the lsit of filenames for the dataframes. 


In [22]:
#@title Data_imports and Dataframe creation { form-width: "30px" }
#@markdown Create list of files and filter out files by part of filename (in my case \_params_\)

"Create filelist"
#Change directory (Filepath in the /something/ format not \something\format)
directory='/content/drive/My Drive/Pulse_colab_14_07_20/PULSE_EXAMPLE_DATA' #@param {type:"string"} #location files
os.chdir (directory)
#filelist
files=glob.glob('*.dat') #list all files that end in .dat

# "Filter"
filter_out='_params_'#@param {type:"string"} #Getting parameter to FILTER OUT!
FILES=[x for x in files if not re.findall(filter_out, x)] #https://stackoverflow.com/questions/12709062/python-lambda-with-if-but-without-else


Pulse=Clean(file_list=FILES)
Pulse.file_list

['C2_EXAMPLE_RAD_PULSE.dat']

#Check Pulse Values

The pulse measurment is done under the User tab. 


For the various tab you can note:
* cbDoUser = True - meaning the measurment was done from User. 
* cbLeaveChannelOpen = True - Allows transitor to discharge and stabilize Vth
* tbMinIRange = 100e-9 -this shoudl be lowered to get better leakage measurment 

* NumberOfMeasurementCycles= 2000 this can be high so that measurment continues until you press "interupt measurment". 
* CycleDelay= 5 time between each cycle.


In the user tab, you shoudl check for: 

* V2= 10 V2 is VDS
* DV1List	= 0;2;4;6;8. - these are the values it will measure, i dont know why but it is taking small steps between them. (ask Tobias) 
* IniDelay = 100 time before it starts measurment.
* ApertureTime	= 0.2 - in [ms], very short time to not disturb sensor.
* DeltaT = 1 [ms] total open time (1/0.2=5 points per DV1List values) 
* VRangeCh1 =	20.0 It sets to 2 sometimes, it needs to be higher than max(DV1List)
* VRangeCh2 =	20.0 it needs to be higher than V2
* ComplianceCh1/2	= 1e-3 this protects the machine and sample from damage. If you see your current/VG drop it has gone over compliance - this is probably due to shorting with the tips or a broken TFT. 



In [23]:
#@title create param_df and show various/user tab { form-width: "30px" }
param_df=pd.read_csv('C2_EXAMPLE_RAD_PULSE_params_.dat', sep=':', names=['Empty','Tab','Param','Value'], header=None).drop('Empty', axis=1)
param_df[param_df['Tab'].isin(['various', 'user'])]



,Tab,Param,Value
2,various,cbSingleMeasure,True
3,various,cbDoScan,False
4,various,cbDoOsci,False
5,various,cbDoUser,True
6,various,cbDo4P,False
7,various,cbLeaveChannelOpen,True
8,various,cbOverflowCheck,False
9,various,cbCh1Floating,True
10,various,cbCh2Floating,True
11,various,tbMinIRange,100e-9


In [24]:
#@title Visualize important parameters { form-width: "30px" }
important_param_Pulse = ['cbDoUser','cbLeaveChannelOpen','tbMinIRange','NumberOfMeasurementCycles','CycleDelay','V2','DV1List','IniDelay','ApertureTime','DeltaT','VRangeCh1','VRangeCh2','ComplianceCh1','ComplianceCh2' ]
param_df[(param_df['Param'].isin(important_param_Pulse)) & (param_df['Tab'].isin(['various', 'user'])) ] 


,Tab,Param,Value
5,various,cbDoUser,True
7,various,cbLeaveChannelOpen,True
11,various,tbMinIRange,100e-9
14,various,NumberOfMeasurementCycles,2000
15,various,CycleDelay,5
55,user,V2,10
56,user,DV1List,0;2;4;6;8
58,user,IniDelay,100
59,user,ApertureTime,0.2
60,user,DeltaT,1


## Visualization of scan parameters 

Here is the visuallization of the scan parameters. 

Comparing to the FIG2A in the paper: 

*  In the paper VG increases in steps, collecting multiple points at the same VG value. In my setup it incraeases with every point but 2
* In the paper maxVG=5, for me MaxVG=8 




---


Paper: Pulse characteristics aquistion: Gate voltage and drain current as a function of time

![Pulse characteristics aquistion: Gate voltage and drain current as a function of time](https://drive.google.com/uc?export=view&id=1uQC4zNGIE53Osy43_uGMd2u1yjvuZtfS)


In [25]:
#@title Supporting material: Pulse param - VG/VD/absID as a function fo time { form-width: "30px" }

df=Pulse.clean[0] #choosing df to be this particular dataframe cause it works out well. obviously change df with pulse.clean[whatever]when using it 
Plot_Pulse_param(df[df['iterations'].isin([1])]) #isin([1]) takes the first iteration



#Extraction of Values

## Extraction of the Vth

Vth indicates the value to overcome from the gate electrode to achieve
an accumulation region in the interface between semiconductor and dielectric
layer. This value can be evaluated by a linear extrapolation in the ID-VG
plot (for low VD) or in the $ID^{0.5}$ - VG Plot (for high VD)

---

Since VD= 10 (high)

Vth is extracted by doing:
1. >$\left | \sqrt{ID} \right |>0.002$ limit data to linear region (CRAMER RULE)

2. >$linregression(VG, \left | \sqrt{ID} \right |)$ get slope, y_intercept

3. >$Vth[V] = -intercept/slope$ 



---

Here is the result from FIG2B in the paper:  Transfer characteristics obtained from the fast transfer in saturation and line
fit to obtain Vth.

![Transfer characteristics obtained from the fast transfer in saturation and line
fit to obtain Vth.](https://drive.google.com/uc?export=view&id=1qVWNnNjIw8A9nGddY-mgtLynUSFhGjt0)





In [26]:
#@title Supporting material: VTH extraction from pulse
Plot_Pulse_VTh_extraction(df[df['iterations'].isin([1])])


## Extraction of Capacitance

I used the plot in Fig3A of the Paper to estimate capacitance. 

![Capacitance extraction](https://drive.google.com/uc?export=view&id=1yRBzQTZVWtxcJ9NRr2E_t5y28NKweVba)  


For Batch1, d=250[nm], 1/c=30;
>  $c=33 nF/cm^{2}$


## Extraction of Fied effect mobility in Saturation mode

To extract sat_mobility i used equation: 
> $ \mu_{sat} = \frac{2L}{WC}(\frac{\delta {\sqrt{\left |I_{DS} \right |}}}{\delta{V_{GS}}})^2 = \frac{cm^2}{V*s} $

Where C is capacitance, W/L are the transitor's dimensions. 


From a bit of research, i am not sure this is the right equation. 

---

From [wikipedia electron mobility](https://en.wikipedia.org/wiki/Electron_mobility#Electric_field_dependence_and_velocity_saturation): 
Using saturation mode
In this technique,[20] for each fixed gate voltage VGS, the drain-source voltage VDS is increased until the current ID saturates. Next, the square root of this saturated current is plotted against the gate voltage, and the slope msat is measured. Then the mobility is

> $\mu =m_{sat}^{2}{\frac {2L}{W}}{\frac {1}{C_{i}}}$

This matches  my equation. 

---

From Researcgate Q:[ How_to_determine_the_value_of_effective_hole_and_electron_mobility_from_graph_of_square_root_drain_current_vs_gate_voltage](https://www.researchgate.net/post/How_to_determine_the_value_of_effective_hole_and_electron_mobility_from_graph_of_square_root_drain_current_vs_gate_voltage)


it is very easy. You want to determine the value of mobility from the graph of square root drain current vs gate voltage - it means you want to find field-effect mobility in the  saturation region.
1. Plot the graph of square root drain current vs gate voltage.
2. Fit a line to the most linear region of your curve (i.e. do not include data below threshold voltage).
3. Once you have a fit in the form y=ax + b, extract a and insert this value to the following formula:
mobility = 2L/W x 1/Ci x (a x a)
You can obtain this formula calculating derivative d[sqrt(Isd]/d(Vg) using the equation for Isd in the saturation region.
4. Check your units and if everything is OK you should get your saturation region mobility in cm2/Vs.

---

However Ilaria in the Thesis indicates an equation with VD: 

Saturation mobility (µsat) Obtained with high VD:
> $ \mu_{sat} = \frac{(\frac{\delta {\sqrt{I_{DS}}}}{\delta{V_{GS}}})^2} {\frac{1}{2}C_i \frac{W}{L}V_D} $


 (1.7)
This mobility doesn’t require Vth and it is less sensitive to the contact
resistance but it describes a situation where the channel is pinched-off
and its effective length is smaller than L.

---

In her equation there is VD present, while in the one that i used no. This could have major differences on our mobility comparisons. 



In [27]:
#@title Mobility in cm^2/(V*s)
mob_sat(df['#VGate(V)'],df['IDrain(A)'],C=3.3e-8, W=0.016, L=0.002)

14.062471170454263

## Extraction of On/Off

On/off ratio This parameter defines the ratio between the maximum and the
minimum value of the drain current. A large value is desirable to obtain a
good device for successful usage as electronic switch.

In [28]:
#@title on_off extraction 
on_off( np.max(abs(df['IDrain(A)'])),np.min(abs(df['IDrain(A)'])))


12496799999.999998

##Extraction of Subthreshold Swing 

Subthreshold swing (S) S is defined as the inverse of the maximum slope of
the transfer characteristic. It represents the necessary VG to increase ID by
one decade

 > $S [V/dec] = \frac{1}{max\frac{\delta \log{I_{DS}}}{{\delta{V_{GS}}}}}$


 


In [29]:
#@title Extraction of Subthreshold Swing { form-width: "30px" }
subthreshold_swing(df['#VGate(V)'], abs(df['IDrain(A)']))

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:1269: RuntimeWarning:

invalid value encountered in subtract



1.2099521305629775e-06

# Plots for presentation 

Important Plots are VTh vs time, Mob vs time, 

In [15]:
#@title Create the dataframe of iteration, time, mob, Vth
df_values= Vth_mob_extraction_vs_iteration(Pulse.clean[0])
df_values.head()

,iteration,time(s),Vth_sat,Mob_sat
0,1.0,0.132,0.474591,8.210427
1,2.0,5.187,0.475033,8.211992
2,3.0,10.252,0.475966,8.212978
3,4.0,15.326,0.476044,8.213401
4,5.0,20.406,0.476562,8.215133


In [16]:
#@title PLOT: Vth-Mob vs time { form-width: "30px" }
#Create the plot Mob-Vth vs time
Plot_Pulse_param(df_values)

KeyError: ignored

In [ ]:

Iter_Rad_Start = 10 #@param {type:"integer"}
Iter_Rad_Stop = 8 #@param {type:"integer"}



In [ ]:
#@title Ilaria Graph: Vth diff vs time { form-width: "30px" }



Rad_start=[594,611,627,643,658,674,690]
Rad_stop= [603,618,635,651,667,681,698]
Vth_initial=df_values['Vth_sat'].iloc[Rad_start[0]]
Vth_0=df_values['Vth_sat'].iloc[Rad_stop[-1]]


df=df_values
df['Vth-norm']=(df["Vth_sat"]-Vth_initial) / abs(Vth_initial-Vth_0)
df['time(s)']=df["time(s)"]-df['time(s)'].iloc[Rad_stop[-1]]
Vth_recovered = next(x[0] for x in enumerate(df['Vth-norm'].iloc[Rad_stop[-1]:]) if x[1] > 0)
print (Vth_recovered)
fig = make_subplots(specs=[[{"secondary_y": True}]])

  # Vth vs time 
fig.add_trace(
      go.Scatter(x=df['time(s)'], y=df["Vth-norm"], name='Vth normalize after 180mGy', line=dict(color='orange' ), hovertext=df['iteration']),
      secondary_y=False
  )


# Set x,y-axis title
fig.update_xaxes(title_text="Time[s]", titlefont=dict(size=20))
fig.update_yaxes( title_text="Normalized Vth [V]", secondary_y=False, titlefont=dict(size=20))
# fig.update_yaxes( title_text="Time[s]", secondary_y=True, titlefont=dict(size=20))

fig.update_layout(
    width = 800,
    height = 500,
    title = "fixed-ratio axes with compressed axes",
    xaxis = dict(
      range=[0,3000],  # sets the range of xaxis
        # meanwhile compresses the xaxis by decreasing its "domain"
    ),
   
)

fig.show()


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(
      go.Scatter(x=df_values["time(s)"]-df_values['time(s)'].iloc[Rad_stop[-1]], y=df_values["Vth-norm"], name='Vth normalize after 180mGy', line=dict(color='orange' )),
      secondary_y=False
  )
# fig.update_layout(
#     width = 800,
#     height = 500,
#     title = "fixed-ratio axes with compressed axes",
#     xaxis = dict(
#       range=[0,1000],  # sets the range of xaxis
#       constrain="domain",  # meanwhile compresses the xaxis by decreasing its "domain"
#     ),
#     yaxis = dict(
#       scaleanchor = "x",
#       scaleratio = 1,
#     ),
# )
fig.show()